In [5]:
pip install gspread oauth2client

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: /Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [28]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials

def find_unique_classes_with_failures_or_errors(sheet_url):
    try:
        # Connect to Google Sheets API
        scope = ['https://spreadsheets.google.com/feeds', 'https://www.googleapis.com/auth/drive']
        creds = ServiceAccountCredentials.from_json_keyfile_name('credentials.json', scope)
        client = gspread.authorize(creds)
        
        # Open the Google Sheets document
        sheet = client.open_by_url(sheet_url)
        worksheet = sheet.get_worksheet(0)  # Assuming data is in the first worksheet
        
        # Get all values from the sheet
        all_data = worksheet.get_all_records()
        
        # Find unique 'Class Name' columns for which either 'Failure_C' or 'Errors_C' is greater than 0
        unique_classes = set()
        for row in all_data:
            if row['Failure_C'] > 0 or row['Errors_C'] > 0:
                unique_classes.add(row['Class Name'])
        
        if unique_classes:
            print("Unique 'Class Name' columns for which either 'Failure_C' or 'Errors_C' is greater than 0:")
            # for class_name in unique_classes:
            #     print(class_name)
            return unique_classes
        else:
            print("No unique 'Class Name' columns found for which either 'Failure_C' or 'Errors_C' is greater than 0.")
            return []
    except Exception as e:
        print(f"An error occurred: {e}")

# Example usage
google_sheets_url = 'https://docs.google.com/spreadsheets/d/1kMP-sG4JUw97mLs_fwXse3vZta2YNItA4n7MmBLnUSg/edit#gid=952791274'



In [47]:
def write_tests_in_class(tests_in_class, file_name):
    """
    Generate a Markdown file with the provided row values.
    
    Args:
    - row_values: List containing the values for a single row.
    - file_name: Name of the file to be generated.
    """
    with open(file_name, 'w') as f:
        for test in tests_in_class:
            f.write("| Module Path | Class Name | Test Name | Failures | Errors |\n")
            f.write("| :----------- | :--------- | :-------- | :------- | :----- |\n")
            f.write(f"| {test[0]} | {test[1]} | {test[2]} | {test[3]} | {test[4]} |\n")
            f.write("\n")
            f.write("**Reason(s) for Failure(s):**")
            f.write("\n" *2)
            f.write("\n")
            f.write("**Potential fixes:**")
            f.write("\n"*10)
            f.write("<br>"*2)
            f.write("\n")
            f.write("________")
            f.write("\n")

def get_tests_info(sheet_url, unique_class_names):
    try:
        # Connect to Google Sheets API
        scope = ['https://spreadsheets.google.com/feeds', 'https://www.googleapis.com/auth/drive']
        creds = ServiceAccountCredentials.from_json_keyfile_name('credentials.json', scope)
        client = gspread.authorize(creds)
        
        # Open the Google Sheets document
        sheet = client.open_by_url(sheet_url)
        existing_worksheet = sheet.get_worksheet(0)  # Assuming data is in the first worksheet
        
        # Get all values from the existing sheet
        all_data = existing_worksheet.get_all_records(empty2zero=True, head=1)

        for class_ in unique_class_names:
            tests_in_class = [row for row in all_data if row['Class Name'] == class_]
            tests_in_class = sorted(tests_in_class, key=lambda x: x['Failure_C'] + x['Errors_C'], reverse=True)
            if tests_in_class is None:
                print("No data found for the provided class names.")
                return
            
            # format [['Module Path', 'Class Name', 'TestName', 'Failure_C', 'Errors_C']]
            data_to_write = []
            for row in tests_in_class:
                data_to_write.append([row['Module Path'], row['Class Name'], row['TestName'], row['Failure_C'], row['Errors_C']])
            write_tests_in_class(data_to_write, class_+".md") 
        
        print("{} MD files have been generated".format(len(unique_class_names)))
    except Exception as e:
        print(f"An error occurred: {e}")

google_sheets_url = 'https://docs.google.com/spreadsheets/d/1kMP-sG4JUw97mLs_fwXse3vZta2YNItA4n7MmBLnUSg/edit#gid=952791274'
unique_classes = uniqueClasses = find_unique_classes_with_failures_or_errors(google_sheets_url)
get_tests_info(google_sheets_url, unique_classes)

Unique 'Class Name' columns for which either 'Failure_C' or 'Errors_C' is greater than 0:
25 MD files have been generated
